In [1]:


import pandas as pd
from src.data.ingestion import load_tickets

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

In [3]:
json_data = load_tickets("../data/raw/support_tickets.json")

In [4]:
json_data[:5]

([Ticket(ticket_id='TK-2024-000001', created_at=datetime.datetime(2023, 11, 2, 12, 30, 10, tzinfo=datetime.timezone.utc), updated_at=datetime.datetime(2023, 11, 2, 15, 30, 46, tzinfo=datetime.timezone.utc), customer_id='CUST-02387', customer_tier='starter', organization_id='ORG-234', product='CloudBackup Enterprise', product_version='4.5.10', product_module='encryption_layer', category='Feature Request', subcategory='Documentation', priority='critical', severity='P2', channel='portal', subject='Request: Add bulk operation support to CloudBackup Enterprise', description='We would like to request a feature for CloudBackup Enterprise that allows bulk operations. Currently, we have to process items one by one, which is time-consuming. Having bulk support would greatly improve our workflow efficiency.', error_logs='', stack_trace='', customer_sentiment='frustrated', previous_tickets=9, resolution='Issue resolved by updating configuration settings. Changed timeout values from 30s to 120s in 

In [63]:
import pandas as pd

df_data = pd.DataFrame([ticket.model_dump() for ticket in json_data[0]])
df_data.head()


,ticket_id,created_at,updated_at,customer_id,customer_tier,organization_id,product,product_version,product_module,category,...,escalated,transferred_count,satisfaction_score,resolution_helpful,tags,environment,business_impact,affected_users,language,region
0,TK-2024-000001,2023-11-02 12:30:10+00:00,2023-11-02 15:30:46+00:00,CUST-02387,starter,ORG-234,CloudBackup Enterprise,4.5.10,encryption_layer,Feature Request,...,True,0,4,True,"[error, api, integration, timeout, bug]",production,high,222,de,APAC
1,TK-2024-000002,2023-02-10 16:31:31+00:00,2023-02-12 09:59:43+00:00,CUST-03724,free,ORG-435,DataSync Pro,4.1.11,data_validator,Account Management,...,True,3,4,True,"[database, bug, authentication, data, error]",production,medium,18,ja,MEA
2,TK-2024-000003,2024-09-30 07:43:47+00:00,2024-09-30 11:58:47+00:00,CUST-00600,enterprise,ORG-208,API Gateway,3.1.4,request_router,Feature Request,...,False,3,4,True,"[configuration, error, sync, performance]",staging,medium,591,ja,NA
3,TK-2024-000004,2024-11-27 18:17:26+00:00,2024-11-30 22:07:50+00:00,CUST-04795,starter,ORG-231,CloudBackup Enterprise,3.4.15,backup_service,Account Management,...,False,2,3,False,"[authentication, api, performance]",production,critical,34,en,LATAM
4,TK-2024-000005,2024-03-09 15:41:02+00:00,2024-03-10 10:53:38+00:00,CUST-01101,starter,ORG-241,StreamProcessor,2.8.8,monitoring,Feature Request,...,False,2,5,True,"[data, integration, security, authentication]",development,medium,325,de,MEA


In [64]:
df_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110000 entries, 0 to 109999
Data columns (total 35 columns):
 #   Column              Non-Null Count   Dtype              
---  ------              --------------   -----              
 0   ticket_id           110000 non-null  object             
 1   created_at          110000 non-null  datetime64[ns, UTC]
 2   updated_at          110000 non-null  datetime64[ns, UTC]
 3   customer_id         110000 non-null  object             
 4   customer_tier       110000 non-null  object             
 5   organization_id     110000 non-null  object             
 6   product             110000 non-null  object             
 7   product_version     110000 non-null  object             
 8   product_module      110000 non-null  object             
 9   category            110000 non-null  object             
 10  subcategory         110000 non-null  object             
 11  priority            110000 non-null  object             
 12  severity        

In [65]:
df_data['category'].unique()

array(['Feature Request', 'Account Management', 'Security', 'Data Issue',
       'Technical Issue'], dtype=object)

In [66]:
df_data[df_data['category'].isnull()]

,ticket_id,created_at,updated_at,customer_id,customer_tier,organization_id,product,product_version,product_module,category,...,escalated,transferred_count,satisfaction_score,resolution_helpful,tags,environment,business_impact,affected_users,language,region


In [67]:
df_data['subject'].nunique()

110

In [68]:
df_data['description'].nunique()

350

In [69]:
df_data['category'].value_counts()

category
Security              22085
Data Issue            22050
Feature Request       22047
Account Management    21997
Technical Issue       21821
Name: count, dtype: int64

In [70]:
df_data['subcategory'].unique()

array(['Documentation', 'Upgrade', 'New Feature', 'API', 'Compliance',
       'Corruption', 'Enhancement', 'Sync Error', 'UI/UX',
       'Import/Export', 'Bug', 'Access Control', 'Vulnerability',
       'Authentication', 'Billing', 'Integration', 'Configuration',
       'Performance', 'Encryption', 'License', 'Authorization',
       'Compatibility', 'Subscription', 'Validation', 'Data Loss'],
      dtype=object)

In [71]:
from typing import List, Dict, Any, Tuple

def prepare_classification_dataset(records: List[Dict[str, Any]]) -> Tuple[List[Dict[str, Any]], List[str]]:
    data = []
    for r in records:
        if hasattr(r, "model_dump"):
            r = r.model_dump()
        elif not isinstance(r, dict):
            r = dict(r)
        
        #text = f"{r.get('subject','')} \n {r.get('description','')} \n {r.get('error_logs','')}"
        #text = f"{r.get('subject','')} \n  {r.get('error_logs','')}"

        data.append({
            "subject": r.get("subject", ""),
            "description": r.get("description", ""),
            "error_logs": r.get("error_logs", ""),
            # a few easy metadata signals
            "product": r.get("product", ""),
            "product_module": r.get("product_module", ""),
            "priority": r.get("priority", ""),
            "channel": r.get("channel", ""),
            "customer_tier": r.get("customer_tier", ""),
            "region": r.get("region", ""),
            "category": r.get("category","")
        })
        
    return data

In [72]:
classification_data= prepare_classification_dataset(json_data[0])
classification_data = pd.DataFrame(classification_data)
classification_data.head()

,subject,description,error_logs,product,product_module,priority,channel,customer_tier,region,category
0,Request: Add bulk operation support to CloudBa...,We would like to request a feature for CloudBa...,,CloudBackup Enterprise,encryption_layer,critical,portal,starter,APAC,Feature Request
1,License upgrade needed for DataSync Pro,We need to upgrade our license for DataSync Pr...,2023-02-10T16:31:31 WARN Rate limit approachin...,DataSync Pro,data_validator,medium,chat,free,MEA,Account Management
2,Request: Add bulk operation support to API Gat...,We would like to request a feature for API Gat...,2024-09-30T07:43:47 DEBUG Processing request I...,API Gateway,request_router,high,phone,enterprise,NA,Feature Request
3,License upgrade needed for CloudBackup Enterprise,We need to upgrade our license for CloudBackup...,2024-11-27T18:17:26 ERROR ERROR_SERVER_500: Co...,CloudBackup Enterprise,backup_service,low,portal,starter,LATAM,Account Management
4,Request: Add bulk operation support to StreamP...,We would like to request a feature for StreamP...,2024-03-09T15:41:02 ERROR ERROR_RATELIMIT_429:...,StreamProcessor,monitoring,high,slack,starter,MEA,Feature Request


In [73]:
classification_data = classification_data.drop_duplicates(['subject','description','error_logs'],keep='first')


In [74]:
import pandas as pd
from sklearn.model_selection import GroupShuffleSplit

df = classification_data.copy()
df = df[:10000]
# 1) Create a group id so duplicates are kept together
#df["text_key"] = (df["subject"].fillna("") + " || " + df["description"].fillna("")).astype(str)
df["text_key"] = (df["description"].fillna("")).astype(str)

df["group_id"] = pd.util.hash_pandas_object(df["text_key"], index=False)
df["text"] = df["subject"] + df["description"] + df["error_logs"]

X = df.drop(columns=["category","subject","description","error_logs"]) 
y = df["category"]
groups = df["group_id"]

# 2) Split 70/30 by groups (no duplicate leakage)
gss = GroupShuffleSplit(n_splits=1, test_size=0.30, random_state=42)
train_idx, tmp_idx = next(gss.split(X, y, groups=groups))

X_train, y_train = X.iloc[train_idx], y.iloc[train_idx]
X_tmp, y_tmp  = X.iloc[tmp_idx], y.iloc[tmp_idx]
groups_tmp = groups.iloc[tmp_idx]

# 3) Split remaining 30 into 15/15 by groups
gss2 = GroupShuffleSplit(n_splits=1, test_size=0.50, random_state=44)
val_idx_rel, test_idx_rel = next(gss2.split(X_tmp, y_tmp, groups=groups_tmp))

X_val, y_val = X_tmp.iloc[val_idx_rel], y_tmp.iloc[val_idx_rel]
X_test, y_test = X_tmp.iloc[test_idx_rel], y_tmp.iloc[test_idx_rel]


In [75]:
classification_data.shape

(76944, 10)

In [76]:
# for col in ['product', 'product_module', 'priority', 'channel',
#        'customer_tier', 'region',]:
#     print(f"Unique categories in {col} feature: {classification_data[col].unique()}\n")



In [77]:
X_train.shape

(7376, 9)

In [78]:
# # split: train/val/test = 70/15/15 (via 70/30 then 50/50 of remaining)
# X_train, X_tmp, y_train, y_tmp = train_test_split(
#     classification_data.drop('category',axis=1), classification_data['category'], test_size=0.30, random_state=42, stratify=classification_data['subject','description']
# )
# X_val, X_test, y_val, y_test = train_test_split(
#     X_tmp, y_tmp, test_size=0.50, random_state=42, stratify=y_tmp
# )

In [79]:
X_train.head()

,product,product_module,priority,channel,customer_tier,region,text_key,group_id,text
1,DataSync Pro,data_validator,medium,chat,free,MEA,We need to upgrade our license for DataSync Pr...,9757097747101013450,License upgrade needed for DataSync ProWe need...
2,API Gateway,request_router,high,phone,enterprise,NA,We would like to request a feature for API Gat...,6757513138608122457,Request: Add bulk operation support to API Gat...
3,CloudBackup Enterprise,backup_service,low,portal,starter,LATAM,We need to upgrade our license for CloudBackup...,9308935704265343520,License upgrade needed for CloudBackup Enterpr...
5,StreamProcessor,batch_processor,low,email,premium,NA,We have concerns about the authentication mech...,8505384652536865832,Security concern with StreamProcessor authenti...
7,API Gateway,cache_layer,critical,api,premium,NA,We would like to request a feature for API Gat...,6757513138608122457,Request: Add bulk operation support to API Gat...


In [80]:
X_train['text'].nunique()

7376

In [81]:
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder

TEXT_COL = "text"
CAT_COLS = [
    "product",
    "product_module",
    "priority",
    "channel",
    "customer_tier",
    "region",
]

preprocess = ColumnTransformer(
    transformers=[
        (
            "text",
            TfidfVectorizer(
                max_features=80_000,
                ngram_range=(1, 2),
                min_df=2,
            ),
            TEXT_COL,   # pandas column name
        ),
        (
            "cat",
            OneHotEncoder(
                handle_unknown="ignore",
                
            ),
            CAT_COLS,   # list of pandas columns
        ),
    ],
    remainder="drop"   # ignore other dataframe columns
)


In [82]:
preprocess_pipe = Pipeline([
    ("preprocess", preprocess)
])

X_transformed = preprocess_pipe.fit_transform(X_train)
X_val_transformed = preprocess_pipe.transform(X_val)
X_test_transformed = preprocess_pipe.transform(X_test)



In [83]:
X_train.shape

(7376, 9)

In [84]:
print(X_transformed.shape)


(7376, 7684)


In [85]:
#preprocess_pipe.get_feature_names_out

In [86]:
logistric_model = LogisticRegression(
        max_iter=2000,
        n_jobs=None,          # keep portable across OS
        class_weight="balanced"  # helps if categories are imbalanced
    )

logistric_model.fit(X=X_transformed,y=y_train)
# Validation
val_pred = logistric_model.predict(X_val_transformed)
val_f1 = f1_score(y_val, val_pred, average="weighted")
print("\n--- VALIDATION ---")
print("Weighted F1:", round(val_f1, 4))
print(classification_report(y_val, val_pred, digits=3))

# Test
test_pred = logistric_model.predict(X_test_transformed)
test_f1 = f1_score(y_test, test_pred, average="weighted")
print("\n--- TEST ---")
print("Weighted F1:", round(test_f1, 4))
print(classification_report(y_test, test_pred, digits=3))



--- VALIDATION ---
Weighted F1: 1.0
                 precision    recall  f1-score   support

     Data Issue      1.000     1.000     1.000       369
       Security      1.000     1.000     1.000       215
Technical Issue      1.000     1.000     1.000       319

       accuracy                          1.000       903
      macro avg      1.000     1.000     1.000       903
   weighted avg      1.000     1.000     1.000       903


--- TEST ---
Weighted F1: 1.0
                 precision    recall  f1-score   support

     Data Issue      1.000     1.000     1.000       205
Feature Request      1.000     1.000     1.000       791
       Security      1.000     1.000     1.000       425
Technical Issue      1.000     1.000     1.000       300

       accuracy                          1.000      1721
      macro avg      1.000     1.000     1.000      1721
   weighted avg      1.000     1.000     1.000      1721



In [87]:
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix, hstack

from sentence_transformers import SentenceTransformer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.metrics import classification_report, f1_score

from xgboost import XGBClassifier

In [95]:
def encode_text(model: SentenceTransformer, texts: list[str], batch_size: int = 64) -> np.ndarray:


    emb = model.encode(
        texts,
        batch_size=batch_size,
        show_progress_bar=True,
        convert_to_numpy=True,
        normalize_embeddings=True,
           device='cpu'   # often improves downstream linear/tree models
    )
    return emb



def encode_text_parallel(
    model: SentenceTransformer,
    texts: list[str],
    batch_size: int = 256,
) -> np.ndarray:
    """
    Encode texts in parallel using multiple CPU processes.

    Notes:
    - Uses all available CPU cores by default
    - Much faster than single-process encode() on large datasets
    """

    # Start multiprocessing pool
    pool = model.start_multi_process_pool()

    try:
        emb = model.encode_multi_process(
            texts,
            pool,
            batch_size=batch_size,
            normalize_embeddings=True,
            show_progress_bar=True,
        )
    finally:
        # Always clean up processes
        model.stop_multi_process_pool(pool)

    return emb


In [96]:
# --- label encoding ---
le = LabelEncoder()
y_train_enc = le.fit_transform(y_train)
y_val_enc = le.transform(y_val)
y_test_enc = le.transform(y_test)


# --- text embeddings ---
embedder = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

X_train_text = encode_text(embedder, X_train["text"].tolist(), batch_size=256)
X_val_text   = encode_text(embedder, X_val["text"].tolist(), batch_size=64)
X_test_text  = encode_text(embedder, X_test["text"].tolist(), batch_size=64)

Batches: 100%|██████████| 27/27 [00:30<00:00,  1.13s/it]


In [97]:
# Convert dense embeddings to sparse so we can hstack with OneHot output
X_train_text_sp = csr_matrix(X_train_text)
X_val_text_sp   = csr_matrix(X_val_text)
X_test_text_sp  = csr_matrix(X_test_text)

# --- categorical one-hot ---
ohe = OneHotEncoder(handle_unknown="ignore")
X_train_cat = ohe.fit_transform(X_train[CAT_COLS])
X_val_cat   = ohe.transform(X_val[CAT_COLS])
X_test_cat  = ohe.transform(X_test[CAT_COLS])

# --- combine ---
X_train_final = hstack([X_train_text_sp, X_train_cat], format="csr")
X_val_final   = hstack([X_val_text_sp,   X_val_cat],   format="csr")
X_test_final  = hstack([X_test_text_sp,  X_test_cat],  format="csr")

In [98]:
# --- XGBoost ---
xgb = XGBClassifier(
    objective="multi:softprob",
    num_class=len(le.classes_),
    n_estimators=500,
    max_depth=6,
    learning_rate=0.08,
    subsample=0.9,
    colsample_bytree=0.9,
    reg_lambda=1.0,
    tree_method="hist",
    eval_metric="mlogloss",
    random_state=42,
)

xgb.fit(X_train_final, y_train_enc)


,"objective objective: typing.Union[str, xgboost.sklearn._SklObjWProto, typing.Callable[[typing.Any, typing.Any], typing.Tuple[numpy.ndarray, numpy.ndarray]], NoneType]Specify the learning task and the corresponding learning objective or a customobjective function to be used.For custom objective, see :doc:`/tutorials/custom_metric_obj` and:ref:`custom-obj-metric` for more information, along with the end note forfunction signatures.",'multi:softprob'
,"base_score base_score: typing.Union[float, typing.List[float], NoneType]The initial prediction score of all instances, global bias.",None
,booster,None
,"callbacks callbacks: typing.Optional[typing.List[xgboost.callback.TrainingCallback]]List of callback functions that are applied at end of each iteration.It is possible to use predefined callbacks by using:ref:`Callback API `... note:: States in callback are not preserved during training, which means callback objects can not be reused for multiple training sessions without reinitialization or deepcopy... code-block:: python for params in parameters_grid: # be sure to (re)initialize the callbacks before each run callbacks = [xgb.callback.LearningRateScheduler(custom_rates)] reg = xgboost.XGBRegressor(**params, callbacks=callbacks) reg.fit(X, y)",None
,colsample_bylevel colsample_bylevel: typing.Optional[float]Subsample ratio of columns for each level.,None
,colsample_bynode colsample_bynode: typing.Optional[float]Subsample ratio of columns for each split.,None
,colsample_bytree colsample_bytree: typing.Optional[float]Subsample ratio of columns when constructing each tree.,0.9
,"device device: typing.Optional[str].. versionadded:: 2.0.0Device ordinal, available options are `cpu`, `cuda`, and `gpu`.",None
,"early_stopping_rounds early_stopping_rounds: typing.Optional[int].. versionadded:: 1.6.0- Activates early stopping. Validation metric needs to improve at least once in every **early_stopping_rounds** round(s) to continue training. Requires at least one item in **eval_set** in :py:meth:`fit`.- If early stopping occurs, the model will have two additional attributes: :py:attr:`best_score` and :py:attr:`best_iteration`. These are used by the :py:meth:`predict` and :py:meth:`apply` methods to determine the optimal number of trees during inference. If users want to access the full model (including trees built after early stopping), they can specify the `iteration_range` in these inference methods. In addition, other utilities like model plotting can also use the entire model.- If you prefer to discard the trees after `best_iteration`, consider using the callback function :py:class:`xgboost.callback.EarlyStopping`.- If there's more than one item in **eval_set**, the last entry will be used for early stopping. If there's more than one metric in **eval_metric**, the last metric will be used for early stopping.",None
,enable_categorical enable_categorical: boolSee the same parameter of :py:class:`DMatrix` for details.,False
,"eval_metric eval_metric: typing.Union[str, typing.List[typing.Union[str, typing.Callable]], typing.Callable, NoneType].. versionadded:: 1.6.0Metric used for monitoring the training result and early stopping. It can be astring or list of strings as names of predefined metric in XGBoost (See:doc:`/parameter`), one of the metrics in :py:mod:`sklearn.metrics`, or anyother user defined metric that looks like `sklearn.metrics`.If custom objective is also provided, then custom metric should implement thecorresponding reverse link function.Unlike the `scoring` parameter commonly used in scikit-learn, when a callableobject is provided, it's assumed to be a cost function and by default XGBoostwill minimize the result during early stopping.For advanced usage on Early stopping like directly choosing to maximize insteadof minimize, see :py:obj:`xgboost.callback.EarlyStopping`.See :doc:`/tutorials/custom_metric_obj` and :ref:`custom-obj-metric` for moreinformation... code-block:: python from sklearn.datasets import load_diabetes fr

In [99]:
# VAL
val_pred_enc = xgb.predict(X_val_final)
val_pred = le.inverse_transform(val_pred_enc)
print("VAL weighted F1:", round(f1_score(y_val, val_pred, average="weighted"), 4))
print(classification_report(y_val, val_pred, digits=3))

# TEST
test_pred_enc = xgb.predict(X_test_final)
test_pred = le.inverse_transform(test_pred_enc)
print("TEST weighted F1:", round(f1_score(y_test, test_pred, average="weighted"), 4))
print(classification_report(y_test, test_pred, digits=3))

VAL weighted F1: 1.0
                 precision    recall  f1-score   support

     Data Issue      1.000     1.000     1.000       369
       Security      1.000     1.000     1.000       215
Technical Issue      1.000     1.000     1.000       319

       accuracy                          1.000       903
      macro avg      1.000     1.000     1.000       903
   weighted avg      1.000     1.000     1.000       903

TEST weighted F1: 0.9965
                 precision    recall  f1-score   support

     Data Issue      1.000     1.000     1.000       205
Feature Request      1.000     0.992     0.996       791
       Security      1.000     1.000     1.000       425
Technical Issue      0.980     1.000     0.990       300

       accuracy                          0.997      1721
      macro avg      0.995     0.998     0.997      1721
   weighted avg      0.997     0.997     0.997      1721



In [8]:
from typing import Any, Dict, List
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, classification_report
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder

def prepare_classification_dataset(records: List[Any]) -> pd.DataFrame:
    data = []
    for r in records:
        if hasattr(r, "model_dump"):     # Pydantic
            r = r.model_dump()
        elif not isinstance(r, dict):
            r = dict(r)

        text = f"{r.get('subject','')}\n{r.get('description','')}\n{r.get('error_logs','')}"

        data.append({
            "text": text,
            "product": r.get("product", ""),
            "product_module": r.get("product_module", ""),
            "priority": r.get("priority", ""),
            "channel": r.get("channel", ""),
            "customer_tier": r.get("customer_tier", ""),
            "region": r.get("region", ""),
            "category": r.get("category", ""),
        })

    return pd.DataFrame(data)

# IMPORTANT: pass the list of records (not json_data[0] unless nested)
classification_data = prepare_classification_dataset(json_data[0])

X = classification_data.drop("category", axis=1)
y = classification_data["category"]

X_train, X_tmp, y_train, y_tmp = train_test_split(
    X, y, test_size=0.30, random_state=42, stratify=y
)
X_val, X_test, y_val, y_test = train_test_split(
    X_tmp, y_tmp, test_size=0.50, random_state=42, stratify=y_tmp
)

preprocess = ColumnTransformer(
    transformers=[
        ("text", TfidfVectorizer(max_features=80_000, ngram_range=(1, 2), min_df=2), "text"),
        ("cat", OneHotEncoder(handle_unknown="ignore"),
         ["product","product_module","priority","channel","customer_tier","region"]),
    ],
)

X_train_t = preprocess.fit_transform(X_train)
X_val_t = preprocess.transform(X_val)
X_test_t = preprocess.transform(X_test)

model = LogisticRegression(max_iter=2000, class_weight="balanced")
model.fit(X_train_t, y_train)

val_pred = model.predict(X_val_t)
print("VAL weighted F1:", round(f1_score(y_val, val_pred, average="weighted"), 4))
print(classification_report(y_val, val_pred, digits=3))

test_pred = model.predict(X_test_t)
print("TEST weighted F1:", round(f1_score(y_test, test_pred, average="weighted"), 4))
print(classification_report(y_test, test_pred, digits=3))


VAL weighted F1: 1.0
                    precision    recall  f1-score   support

Account Management      1.000     1.000     1.000      3299
        Data Issue      1.000     1.000     1.000      3308
   Feature Request      1.000     1.000     1.000      3307
          Security      1.000     1.000     1.000      3313
   Technical Issue      1.000     1.000     1.000      3273

          accuracy                          1.000     16500
         macro avg      1.000     1.000     1.000     16500
      weighted avg      1.000     1.000     1.000     16500

TEST weighted F1: 1.0
                    precision    recall  f1-score   support

Account Management      1.000     1.000     1.000      3300
        Data Issue      1.000     1.000     1.000      3307
   Feature Request      1.000     1.000     1.000      3307
          Security      1.000     1.000     1.000      3313
   Technical Issue      1.000     1.000     1.000      3273

          accuracy                          1.000   

In [5]:
from src.data.graph_rag_ingest import top_solutions_for_issue,build_support_graph




In [9]:
tickets = json_data[0]

In [11]:
graph = build_support_graph(tickets[:20])

In [23]:
support_graph = build_support_graph(tickets[:])

print("nodes:", support_graph.number_of_nodes())
print("edges:", support_graph.number_of_edges())

print(top_solutions_for_issue(support_graph, "Account Management", "Upgrade", top_k=5))

nodes: 110078
edges: 517739
[('DATA_REPAIR', 0.6649484536082474, 388), ('ENVIRONMENT_ISSUE', 0.6240208877284595, 383), ('DUPLICATE', 0.6222826086956522, 368), ('WORKAROUND', 0.6208955223880597, 335), ('CONFIG_CHANGE', 0.6186666666666667, 375)]


In [14]:
support_graph.nodes

NodeView(('ticket:TK-2024-000001', 'product:CloudBackup Enterprise', 'module:encryption_layer', 'issue:Feature Request|Documentation', 'solution:PATCH_APPLIED', 'ticket:TK-2024-000002', 'product:DataSync Pro', 'module:data_validator', 'issue:Account Management|Upgrade', 'solution:FEATURE_ADDED', 'error:ERROR_DISK_FULL', 'ticket:TK-2024-000003', 'product:API Gateway', 'module:request_router', 'issue:Feature Request|New Feature', 'solution:WORKAROUND', 'ticket:TK-2024-000004', 'module:backup_service', 'solution:CONFIG_CHANGE', 'error:ERROR_SERVER_500', 'ticket:TK-2024-000005', 'product:StreamProcessor', 'module:monitoring', 'issue:Feature Request|API', 'solution:DUPLICATE', 'error:ERROR_RATELIMIT_429', 'ticket:TK-2024-000006', 'module:batch_processor', 'issue:Security|Compliance', 'solution:WONT_FIX', 'error:ERROR_VALIDATION', 'ticket:TK-2024-000007', 'product:Analytics Dashboard', 'module:data_aggregator', 'issue:Data Issue|Corruption', 'solution:ENVIRONMENT_ISSUE', 'error:ERROR_INVALID

#### Graph visuvalization

In [15]:
import random
import networkx as nx
import matplotlib.pyplot as plt

def visualize_subgraph(
    support_graph: nx.Graph,
    center_node: str,
    hops: int = 2,
    max_nodes: int = 80,
    seed: int = 42,
):
    if center_node not in support_graph:
        raise ValueError(f"{center_node} not found in graph")

    # collect nodes within N hops (BFS)
    nodes = {center_node}
    frontier = {center_node}
    for _ in range(hops):
        nxt = set()
        for n in frontier:
            nxt.update(support_graph.neighbors(n))
        nodes.update(nxt)
        frontier = nxt

    nodes = list(nodes)

    # cap nodes for readability
    if len(nodes) > max_nodes:
        random.seed(seed)
        nodes = [center_node] + random.sample([n for n in nodes if n != center_node], max_nodes - 1)

    sub = support_graph.subgraph(nodes).copy()

    # layout
    plt.figure(figsize=(14, 9))
    pos = nx.spring_layout(sub, seed=seed, k=0.7)

    # optional: color by "kind" (we don’t set colors explicitly; matplotlib will default)
    kinds = nx.get_node_attributes(sub, "kind")
    kind_values = sorted(set(kinds.values())) if kinds else []
    if kind_values:
        for kind in kind_values:
            kind_nodes = [n for n in sub.nodes if kinds.get(n) == kind]
            nx.draw_networkx_nodes(sub, pos, nodelist=kind_nodes, node_size=700, alpha=0.9)
    else:
        nx.draw_networkx_nodes(sub, pos, node_size=700, alpha=0.9)

    nx.draw_networkx_edges(sub, pos, alpha=0.5, width=1.5)

    # shorter labels
    def short_label(node: str) -> str:
        for prefix, tag in [("ticket:", ""), ("product:", "P:"), ("module:", "M:"), ("issue:", "I:"), ("solution:", "S:"), ("error:", "E:")]:
            if node.startswith(prefix):
                return tag + node[len(prefix):]
        return node

    labels = {n: short_label(str(n)) for n in sub.nodes}
    nx.draw_networkx_labels(sub, pos, labels=labels, font_size=9)

    # show relationship labels if present
    edge_rels = nx.get_edge_attributes(sub, "rel")
    if edge_rels:
        nx.draw_networkx_edge_labels(sub, pos, edge_labels=edge_rels, font_size=8)

    plt.title(f"Graph-RAG subgraph around: {short_label(center_node)} (≤{max_nodes} nodes, {hops}-hop)")
    plt.axis("off")
    plt.show()


In [22]:
# issue_node = "issue:Account Management|Upgrade"
# visualize_subgraph(support_graph, center_node=issue_node, hops=2, max_nodes=80)


In [ ]:
### by using pyvis in HTML 

from pyvis.network import Network
import networkx as nx


def export_interactive_html(
    support_graph: nx.Graph,
    center_node: str,
    hops: int = 2,
    max_nodes: int = 200,
    out_html: str = "graph_rag.html",
):
    # --- collect subgraph ---
    nodes = {center_node}
    frontier = {center_node}
    for _ in range(hops):
        nxt = set()
        for n in frontier:
            nxt.update(support_graph.neighbors(n))
        nodes.update(nxt)
        frontier = nxt

    nodes = list(nodes)
    if len(nodes) > max_nodes:
        nodes = nodes[:max_nodes]

    sub = support_graph.subgraph(nodes).copy()

    # --- create interactive graph ---
    net = Network(height="750px", width="100%", notebook=True, directed=False)
    net.from_nx(sub)

    # --- add relationship labels to edges ---
    edge_lookup = {}
    for u, v, data in sub.edges(data=True):
        rel = data.get("rel")
        if rel:
            edge_lookup[(u, v)] = rel
            edge_lookup[(v, u)] = rel  # undirected graph

    for edge in net.edges:
        key = (edge["from"], edge["to"])
        rel = edge_lookup.get(key)
        if rel:
            edge["label"] = rel
            edge["title"] = rel

    net.show(out_html)
    return out_html


In [19]:
export_interactive_html(
    support_graph,
    center_node="issue:Account Management|Upgrade",
    hops=2,
    out_html="graph_rag.html"
)


graph_rag.html


'graph_rag.html'